## stAA_default application

### main: run_stAA

In [ ]:
import squidpy as sq
import scanpy as sc
from sklearn.metrics.cluster import (
    v_measure_score, homogeneity_score, completeness_score,  silhouette_score, homogeneity_completeness_v_measure, davies_bouldin_score)
from sklearn.metrics.cluster import adjusted_rand_score
import random
import os
import numpy as np
import torch
from train import stAA
from Processdata import Adata2Torch_data, Spatial_Dis_Cal, process_adata, read_data, get_initial_label
import warnings
warnings.filterwarnings("ignore")


seed = 0
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)

gpu_id = 0
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)


def make_dir(directory_path, new_folder_name):
    """Creates an expected directory if it does not exist"""
    directory_path = os.path.join(directory_path, new_folder_name)
    if not os.path.exists(directory_path):
        os.makedirs(directory_path)
    return directory_path


def eval_embedding(pred, embedding=None):
    sc = silhouette_score(embedding, pred, metric='euclidean')
    db = davies_bouldin_score(embedding, pred)
    return sc, db


def run_stAA(adata, n_clusters, graph_mode="knn", cluster_method="mclust",
             refine=True, data_save_path="./", true_labels=None, eval=True):
    # if cluster_method="louvain", "n_clusters" represents the resolution
    adata = process_adata(adata)

    if graph_mode in ["knn", "KNN"]:
        Spatial_Dis_Cal(adata, knn_dis=5, model="KNN")
    else:
        Spatial_Dis_Cal(adata, rad_dis=graph_mode)

    if 'Spatial_Net' not in adata.uns.keys():
        # 验证是否存在Spatial_Net
        raise ValueError(
            "Please Compute Spatial Network using Spatial_Dis_Cal function first!")
    # Process the data
    data = Adata2Torch_data(adata)
    ss_labels = get_initial_label(adata, method=cluster_method,
                                  n_clusters=n_clusters)
    reso = n_clusters
    if cluster_method == "mclust":
        n_clusters = n_clusters
    else:
        n_clusters = len(set(ss_labels))
    model = stAA(input_dim=data.x.shape[1], epochs=1000,
                 hidden_dim=256, embed_dim=128, n_clusters=n_clusters).cuda()
    res = model.train_model(
        data, method=cluster_method, refine=refine,
        position=adata.obsm['spatial'], eval=eval, reso=reso,
        ss_labels=ss_labels, data_save_path=data_save_path,
        labels=true_labels)
    return res



### Processdata

In [ ]:
import pandas as pd
import numpy as np
import sklearn.neighbors
import scipy.sparse as sp
import seaborn as sns
import matplotlib.pyplot as plt
import ot
import torch
from torch_geometric.data import Data
import scanpy as sc
import metis
import squidpy as sq
from sklearn.decomposition import PCA
from torch_geometric.utils import train_test_split_edges


def read_data(dataset, data_path='/home/sda1/'):
    if dataset == "STARmap":
        file_fold = data_path + str(dataset)
        adata = sc.read(file_fold+"/STARmap_20180505_BY3_1k.h5ad")
        adata.var_names_make_unique()
        df_meta = pd.read_table(file_fold + '/Annotation_STARmap_20180505_BY3_1k.txt',
                                sep='\t', index_col=0)
        adata.obs['ground_truth'] = adata.obs["label"]
        adata.obs['Annotation'] = df_meta.loc[adata.obs_names, 'Annotation'].values

    if dataset == "Breast_cancer":
        file_fold = data_path + str(dataset) #please replace 'file_fold' with the download path
        adata = sc.read_visium(file_fold, count_file='filtered_feature_bc_matrix.h5',
                               load_images=True)
        adata.var_names_make_unique()
        df_meta = pd.read_table(file_fold + '/metadata.tsv',sep='\t',
                                index_col=0)
        adata.obs['ground_truth'] = df_meta.loc[adata.obs_names, 'ground_truth'].values

    if dataset == "Mouse_hippocampus":
        adata = sq.datasets.slideseqv2()
        adata.var_names_make_unique()
    
    if dataset in ["Mouse_embryo_E9_E1S1", "Mouse_embryo_E9_E2S1",
                   "Mouse_embryo_E9_E2S2", "Mouse_embryo_E9_E2S3",
                   "Mouse_embryo_E9_E2S4"]:
        file_fold = data_path + str(dataset)
        adata = sc.read(file_fold+"/MOSTA.h5ad")
        adata.var_names_make_unique()
        adata.obs["ground_truth"] = adata.obs["annotation"]
    
    if dataset == "Mouse_olfactory_slide_seqv2":
        adata = sc.read_h5ad(data_path+"/"+dataset+"/tutorial3.h5ad")

    return adata


def process_adata(adata):
    adata.var_names_make_unique()
    #Normalization
    sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=3000) ##3000高变基因；seurat_v3
    sc.pp.normalize_total(adata, target_sum=1e4) ##normalized data
    sc.pp.log1p(adata)  #log-transformed data
    adata = adata[:, adata.var['highly_variable']]
    return adata


def Adata2Torch_data(adata): 
    G_df = adata.uns['Spatial_Net'].copy() 
    spots = np.array(adata.obs_names) 
    spots_id_tran = dict(zip(spots, range(spots.shape[0]))) 
    G_df['Spot1'] = G_df['Spot1'].map(spots_id_tran) 
    G_df['Spot2'] = G_df['Spot2'].map(spots_id_tran) 

    G = sp.coo_matrix((np.ones(G_df.shape[0]), (G_df['Spot1'], G_df['Spot2'])), 
        shape=(adata.n_obs, adata.n_obs))

    G = G + sp.eye(G.shape[0]) 
    edgeList = np.nonzero(G) 

    if type(adata.X) == np.ndarray:
        data = Data(edge_index=torch.LongTensor(np.array(
            [edgeList[0], edgeList[1]])), x=torch.FloatTensor(adata.X))  
    else:
        data = Data(edge_index=torch.LongTensor(np.array(
            [edgeList[0], edgeList[1]])), x=torch.FloatTensor(adata.X.todense())) 
    data = train_test_split_edges(data)
    return data


def Spatial_Dis_Cal(adata, rad_dis=None, knn_dis=None, model='Radius', verbose=True):
    """\
    Calculate the spatial neighbor networks, as the distance between two spots.
    Parameters
    ----------
    adata:  AnnData object of scanpy package.
    rad_dis:  radius distance when model='Radius' 半径
    knn_dis:  The number of nearest neighbors when model='KNN' 邻居个数
    model:
        The network construction model. When model=='Radius', the spot is connected to spots whose distance is less than rad_dis. 
        When model=='KNN', the spot is connected to its first knn_dis nearest neighbors.
    Returns
    -------
    The spatial networks are saved in adata.uns['Spatial_Net']
    """
    assert(model in ['Radius', 'KNN', "BallTree"]) #断言语句，可以用来调试程序。
    if verbose:
        print('------Calculating spatial graph...')
    coor = pd.DataFrame(adata.obsm['spatial']) #Spot 空间坐标
    coor.index = adata.obs.index #df的index改为spot名称
    # coor.columns = ['imagerow', 'imagecol']
    coor.columns = ['Spatial_X', 'Spatial_Y'] #修改df的列名

    if model == 'Radius':
        nbrs = sklearn.neighbors.NearestNeighbors(radius=rad_dis).fit(coor)
        distances, indices = nbrs.radius_neighbors(coor, return_distance=True)
        # Find the neighbors within a given radius of a point or points, 返回每个Spot在给定半径中的邻居个数及距离。
        # distances, indices的rows等于Spot的个数，即每个Spot都对应一个distance 和 index list.
        KNN_list = []
        for spot in range(indices.shape[0]):
            KNN_list.append(pd.DataFrame(zip([spot]*indices[spot].shape[0], indices[spot], distances[spot]))) #每个spot的邻居编号；距离。
    
    if model == 'KNN':
        nbrs = sklearn.neighbors.NearestNeighbors(n_neighbors=knn_dis+1).fit(coor)
        distances, indices = nbrs.kneighbors(coor)
        KNN_list = []
        for spot in range(indices.shape[0]):
            KNN_list.append(pd.DataFrame(zip([spot]*indices.shape[1],indices[spot,:], distances[spot,:])))

    if model == "BallTree":
        from sklearn.neighbors import BallTree
        tree = BallTree(coor)
        distances, ind = tree.query(coor, k=knn_dis+1)
        indices = ind[:, 1:]
        KNN_list=[]

        for spot in range(indices.shape[0]):
            KNN_list.append(pd.DataFrame(zip([spot]*indices.shape[1],indices[spot,:], distances[spot,:])))
        # for node_idx in range(coor.shape[0]):
        #     for j in np.arange(0, indices.shape[1]):
        #         KNN_list.append(node_idx, indices[node_idx][j])

    KNN_df = pd.concat(KNN_list) #变为dataframe格式。
    KNN_df.columns = ['Spot1', 'Spot2', 'Distance']

    Spatial_Net = KNN_df.copy()
    Spatial_Net = Spatial_Net.loc[Spatial_Net['Distance']>0,]
    id_spot_trans = dict(zip(range(coor.shape[0]), np.array(coor.index), )) #构建一个词典，
    Spatial_Net['Spot1'] = Spatial_Net['Spot1'].map(id_spot_trans) #Spot1的编号，e.g. spot1出现几次，表明有几个邻居，在spot2里。
    Spatial_Net['Spot2'] = Spatial_Net['Spot2'].map(id_spot_trans) #Spot2的编号 spot1对应的邻居编号
    if verbose:
        print('The graph contains %d edges, %d spots.' %(Spatial_Net.shape[0], adata.n_obs)) #共多少条边
        print('%.4f neighbors per spot on average.' %(Spatial_Net.shape[0]/adata.n_obs)) #平均每个Spot多少条边 
    adata.uns['Spatial_Net'] = Spatial_Net


def get_initial_label(adata, n_clusters, refine=True, method="mclust"):
    features = adata.X
    if type(features) == np.ndarray:
        features = features
    else:
        features = features.todense()
    features = np.asarray(features)
    pca_input = dopca(features, dim = 20) # dim 10-45 30 is the best
    if method == "mclust":
        pred = mclust_R(embedding=pca_input, num_cluster=n_clusters)
    if method == "louvain":
        adata.obsm["pca"] = pca_input
        sc.pp.neighbors(adata, n_neighbors=50, use_rep="pca")
        sc.tl.louvain(adata, resolution=n_clusters, random_state=0)
        pred=adata.obs['louvain'].astype(int).to_numpy()
    if refine:
        pred = refine_label(pred, adata.obsm["spatial"], radius=60)
    pred = list(map(int, pred))
    return np.array(pred)


def dopca(X, dim=10):
    pcaten = PCA(n_components=dim, random_state=42)
    X_10 = pcaten.fit_transform(X)
    return X_10


def mclust_R(embedding, num_cluster, modelNames='EEE', random_seed=0):
    """\
    Clustering using the mclust algorithm.
    The parameters are the same as those in the R package mclust.
    """
    np.random.seed(random_seed)
    import rpy2.robjects as robjects
    robjects.r.library("mclust")
    import rpy2.robjects.numpy2ri
    rpy2.robjects.numpy2ri.activate()
    r_random_seed = robjects.r['set.seed']
    r_random_seed(random_seed)
    rmclust = robjects.r['Mclust']
    res = rmclust(rpy2.robjects.numpy2ri.numpy2rpy(
        embedding), num_cluster, modelNames)
    mclust_res = np.array(res[-2])

    mclust_res = mclust_res.astype('int')
    # mclust_res = mclust_res.astype('category')
    return mclust_res

def refine_label(label, position, 
                 radius=50):
    new_type = []

    # calculate distance
    distance = ot.dist(position, position, metric='euclidean')

    n_cell = distance.shape[0]

    for i in range(n_cell):
        vec = distance[i, :]
        index = vec.argsort()
        neigh_type = []
        for j in range(1, radius+1):
            neigh_type.append(label[index[j]])
        max_type = max(neigh_type, key=neigh_type.count)
        new_type.append(max_type)

    new_type = [str(i) for i in list(new_type)]
    # adata.obs['label_refined'] = np.array(new_type)

    return new_type

In [ ]:
# read data
file_fold = '/home/tengliu/Paper6-NC/GraphST/data/Tutorial1' #please replace 'file_fold' with the download path
adata = sc.read_visium(file_fold, count_file='filtered_feature_bc_matrix.h5', load_images=True)
adata.var_names_make_unique()
# add ground_truth
df_meta = pd.read_csv(file_fold + '/metadata.tsv', sep='\t')
df_meta_layer = df_meta['layer_guess']
adata.obs['ground_truth'] = df_meta_layer.values

# Run device, by default, the package is implemented on 'cpu'. We recommend using GPU.
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# the location of R, which is necessary for mclust algorithm. Please replace the path below with local R installation path
os.environ['R_HOME'] ="/home/tengliu/miniconda3/envs/R41/lib/R"

In [ ]:
res = run_stAA(adata, n_clusters=7, cluster_method="mclust", refine=False,
               graph_mode=150, eval=True, data_save_path="./",true_labels=adata.obs["ground_truth"])
print(res["embedding"])
print(res["pred_label"])
print(res["ari"])

In [ ]:
features = adata.X
if type(features) == np.ndarray:
    features = features
else:
    features = features.todense()

In [ ]:
type(features)

In [ ]:
Spatial_Dis_Cal(adata, knn_dis=5, model="KNN")

In [ ]:
data = Adata2Torch_data(adata)

In [ ]:
data #[2, 15470]

In [ ]:
data.train_pos_edge_index

In [ ]:
transfrom = RandomLinkSplit(is_undirected=True)
train_data, val_data, test_data = 

In [ ]:

# train model and cluster
res = run_stAA(adata, n_clusters=7,
               cluster_method="mclust", refine=False,
               graph_mode=40, eval=False,
               data_save_path="./")
print(res["embedding"])
print(res["pred_label"])